# Libraries

In [1]:
# IO
import os
import csv
import pathlib
from pathlib import Path
import chardet
import warnings

# Utilities
import numpy as np 
import pandas as pd
import copy
from functools import partial

# Preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from torchtext.vocab import vocab
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Modelling and training
from sklearn.model_selection import StratifiedKFold
import optuna
import torch
import torch.nn as nn
import torch.nn.functional as F

# Evaluation
from sklearn.metrics import (
    roc_curve, auc, roc_auc_score,
    accuracy_score, precision_score, recall_score, 
    f1_score, confusion_matrix
)
from sklearn.preprocessing import label_binarize

warnings.filterwarnings("ignore", category=FutureWarning)

/Users/exterior/miniconda3/envs/project2/lib/python3.12/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/exterior/miniconda3/envs/project2/lib/python3.12/site-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/exterior/miniconda3/envs/project2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarnin

Download `nltk` resources (only needs to run once per machine)

In [2]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/exterior/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/exterior/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Classes

Early stopping class (pytorch does not have a built in early stopping callback)

In [3]:
class EarlyStopping:
    def __init__(self, patience=1, restore_best_weights=True, mode='min'):
        self.patience = patience
        self.restore_best_weights = restore_best_weights
        self.mode = mode  # 'min' for loss, 'max' for accuracy, etc.
        self.best_score = None
        self.counter = 0
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, current_score, model):
        # Determine if the current score is better
        if self.best_score is None:
            self.best_score = current_score
            if self.restore_best_weights:
                self.best_model_state = copy.deepcopy(model.state_dict())
        elif (self.mode == 'min' and current_score < self.best_score) or \
             (self.mode == 'max' and current_score > self.best_score):
            self.best_score = current_score
            self.counter = 0
            if self.restore_best_weights:
                self.best_model_state = copy.deepcopy(model.state_dict())
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

    def restore(self, model):
        if self.restore_best_weights and self.best_model_state:
            model.load_state_dict(self.best_model_state)

In [4]:
class TextDataset(Dataset):
    def __init__(self, df, vocab_inst):
        self.vocab_inst = vocab_inst
        self.texts = df['Sentence'].tolist()
        self.labels = df['Sentiment_encoded'].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoded = torch.tensor(encode_text(self.texts[idx], vocab_inst), dtype=torch.long)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return encoded, label

## Models

In [5]:
class CorpoSpeakDecoder(nn.Module):
    def __init__(
        self, 
        vocab_size, 
        embed_dim=128, 
        lstm_out=196,
        dropout_spatial=0.5, 
        dropout_lstm=0.3,
        dropout_1=0.2,
        dropout_2=0.4, 
        dense_1=100,
        output_classes=3):
        
        super(CorpoSpeakDecoder, self).__init__()
        
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)
        self.spatial_dropout = nn.Dropout2d(p=dropout_spatial)  # Approximate SpatialDropout1D
        self.lstm = nn.LSTM(embed_dim, lstm_out, batch_first=True, dropout=dropout_lstm)
        
        self.dropout_1 = nn.Dropout(p=dropout_1)
        self.dense_1 = nn.Linear(lstm_out, dense_1)
        self.dropout_2 = nn.Dropout(p=dropout_2)
        self.output_layer = nn.Linear(dense_1, output_classes)

    def forward(self, x):
        x = self.embedding(x)                      # (batch, seq_len, embed_dim)
        x = x.permute(0, 2, 1)                     # For Dropout2d: (batch, embed_dim, seq_len)
        x = self.spatial_dropout(x)
        x = x.permute(0, 2, 1)                     # Back to (batch, seq_len, embed_dim)
        
        x, _ = self.lstm(x)                        # LSTM returns (output, (h_n, c_n))
        x = x[:, -1, :]                            # Get the output from the last timestep
        
        x = self.dropout_1(x)
        x = F.relu(self.dense_1(x))
        x = self.dropout_2(x)
        x = self.output_layer(x)
        return F.softmax(x, dim=1)         

In [6]:
class SimpleRNNClassifier(nn.Module):
    def __init__(
        self, 
        max_features, 
        max_length, 
        embedding_dim=16, 
        rnn_units=64, 
        num_classes=3
        ):
        super(SimpleRNNClassifier, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=max_features, embedding_dim=embedding_dim)
        self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=rnn_units, batch_first=True, nonlinearity='tanh')
        self.fc = nn.Linear(rnn_units, num_classes)

    def forward(self, x):
        x = self.embedding(x)                        # shape: (batch_size, max_length, embedding_dim)
        output, hidden = self.rnn(x)                 # hidden: (1, batch_size, rnn_units)
        x = hidden.squeeze(0)                        # shape: (batch_size, rnn_units)
        x = self.fc(x)                               # shape: (batch_size, num_classes)
        x = F.softmax(x, dim=1)                      # shape: (batch_size, num_classes)
        return x

# Functions

### Preprocessing

The encoding of the text file is automatically detected here with some confidence. It could be extracted from terminal using
```
file -I file.txt
```

Terminal method (not os agnostic):

In [7]:
#!file -i ~/Documents/StudyResources/IML/Project/Part2/_data/FinancialPhraseBank-v1.0/Sentences_50Agree.txt

Python method:

In [8]:
def extract_sentences_from_file(filepath):
    """Given an input txt file, extract sentences 
    and associated sentiments

    Args:
        filepath (string): path to string 

    Returns:
        list: list of sentences and sentiments
    """
    sentences = []
    # automatically detect endoding (best guess)
    with open(filepath, 'rb') as file:
        encoding = chardet.detect(file.read())['encoding']
    # read and split
    with open(filepath, 'r', encoding=encoding) as file:
        for line in file:
            line = line.strip()
            if '.@' in line:
                sentence, sentiment = line.rsplit('.@', 1)
                sentence = sentence.strip()
                sentiment = sentiment.strip().lower()
                sentence = fix_common_mojibake(sentence)
                sentences.append((sentence, sentiment))
    return sentences

In [9]:
def txt_to_csv(txt_path, output_csv):
    """Takes a file path as input and generates a .csv file
    which contains sentences and sentiments as columns, extracted
    from the .txt file corresponding to the path

    Args:
        txt_path (string): path to the .txt file
        output_csv (string): path to desired output .csv file
    """
    # Skip processing if CSV already exists
    if os.path.exists(output_csv):
        print(f"{output_csv} already exists. Skipping processing.")
        return

    sentences = extract_sentences_from_file(txt_path)

    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Sentence', 'Sentiment'])  # Header
        writer.writerows(sentences)
    print(f"Processed files and wrote output to {output_csv}")


This function is needed as the text files seem to be corrupted, for example one sentence is:
```
Clothing retail chain Sepp+ñl+ñ 's sales increased by 8 % to EUR 155.2 mn , and operating profit rose to EUR 31.1 mn from EUR 17.1 mn in 2004
```
The characters `+ñ` are a result of choosing the wrong encoding; in fact, here the detected encoding is latin-1, and those characters correspond to `ä` in utf-8. The function below is used to fix these common mistakes

In [10]:
def fix_common_mojibake(text):
    """Function to manually handle encoding errors

    Args:
        text (string): input sentence

    Returns:
        string: output corrected sentence
    """
    replacements = {
        '+ñ': 'ä',
        '+í': 'é',
        '+ô': 'ö',
        '+ü': 'ü'
        }
    for wrong, right in replacements.items():
        text = text.replace(wrong, right)
    return text

This function summarizes the common preprocessing pipeline for natural language models.

In [11]:
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens)

    return processed_text

Function that creates csv files (if not created already) and returns the dataframe extracted from them

In [12]:
def create_df():
    """Import data and creates csv's and pandas dataframes

    Returns:
        pandas dataframes for training and testing
    """
    print("Please select which dataset to use. Data is categorized based on the percentage of agreement in the sentiment estimator.")
    print("Options:")
    print("(1) 50")
    print("(2) 66")
    print("(3) 75")
    print("(4) 100")
    percentage = input()
    if percentage=='50':
        txt_path = RAW_DATA_FOLDER + 'Sentences_50Agree.txt'
        csv_path = DATASET_FOLDER + 'sentences_50.csv'
        txt_to_csv(txt_path, csv_path)
    elif percentage=='66':
        txt_path = RAW_DATA_FOLDER + 'Sentences_66Agree.txt'
        csv_path = DATASET_FOLDER + 'sentences_66.csv'
        txt_to_csv(txt_path, csv_path)
    elif percentage=='75':
        txt_path = RAW_DATA_FOLDER + 'Sentences_75Agree.txt'
        csv_path = DATASET_FOLDER + 'sentences_75.csv'
        txt_to_csv(txt_path, csv_path)
    elif percentage=='100':
        txt_path = RAW_DATA_FOLDER + 'Sentences_AllAgree.txt'
        csv_path = DATASET_FOLDER + 'sentences_100.csv'
        txt_to_csv(txt_path, csv_path)
    else:
        print("The percentage provided is not admitted, skipping.")
        return

    df = pd.read_csv(csv_path)

    le = LabelEncoder()
    le.fit(df['Sentiment'])
    df['Sentiment_encoded'] = le.transform(df['Sentiment'])

    return df, le

In [13]:
def create_dataloaders(train_df, val_df, vocab_inst=None):
    if vocab_inst is None:
        vocab_inst = build_vocab(train_df)

    train_dataset = TextDataset(train_df, vocab_inst)
    val_dataset = TextDataset(val_df, vocab_inst)

    collate_with_vocab = partial(collate_fn, vocab_inst=vocab_inst)

    train_loader = DataLoader(train_dataset, batch_size=32, collate_fn=collate_with_vocab, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, collate_fn=collate_with_vocab)

    return train_loader, val_loader, vocab_inst


The following functions are needed to create token vocabulary and dataloaders

In [14]:
def build_vocab(df):
    counter = Counter()
    for text in df['Sentence']:
        counter.update(preprocess_text(text))

    vocab_inst = vocab(counter, specials=["<pad>", "<unk>"])
    vocab_inst.set_default_index(vocab_inst["<unk>"])
    return vocab_inst

In [15]:
def encode_text(text, vocab_inst):
    tokens = preprocess_text(text)
    return [vocab_inst[token] for token in tokens]

In [16]:
def collate_fn(batch, vocab_inst):
    texts, labels = zip(*batch)
    padded_texts = pad_sequence(texts, batch_first=True, padding_value=vocab_inst["<pad>"])
    labels = torch.stack(labels)
    return padded_texts, labels

## Hyperparameter tuning

TODO
Maybe it's better to feed it the dictionary generated outside of the objective function?

In [17]:
def objective(
    trial, 
    df,
    vocab_inst,
    cv,
    device
    ):
    # Sample hyperparameters
    embed_dim = trial.suggest_int("embed_dim", 64, 256, step=32)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    wd = trial.suggest_float("weight_decay", 1e-4, 1e-2, log=True)
    lstm_out = trial.suggest_int("lstm_out", 64, 256, step=32)
    dropout_spatial = trial.suggest_float("dropout_spatial", 0.1, 0.5, step=0.2)
    dropout_lstm = trial.suggest_float("dropout_lstm", 0.1, 0.5, step=0.2)
    dropout_1 = trial.suggest_float("dropout_1", 0.1, 0.5, step=0.2)
    dropout_2 = trial.suggest_float("dropout_2", 0.1, 0.5, step=0.2)
    dense_1 = trial.suggest_int("dense_1", 64, 256, step=64)

    accuracies = []

    # Cross-validation loop
    for train_idx, val_idx in cv.split(df, df['Sentiment_encoded']):

        early_stop = EarlyStopping(patience=3, mode='max')

        train_df = df.iloc[train_idx].reset_index(drop=True)
        val_df = df.iloc[val_idx].reset_index(drop=True)

        # Build vocab only on training fold
        vocab_size = len(vocab_inst)

        train_loader, val_loader, _ = create_dataloaders(train_df, val_df, vocab_inst=vocab_inst)

        # Define model (custom RNN or LSTM)
        model = CorpoSpeakDecoder(
            vocab_size, 
            embed_dim=embed_dim, 
            lstm_out=lstm_out,
            dropout_spatial=dropout_spatial,
            dropout_lstm=dropout_lstm, 
            dropout_1=dropout_1,
            dropout_2=dropout_2,
            dense_1=dense_1
            )
        model = model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        loss_crit = nn.CrossEntropyLoss().to(device)

        # Train for 5 epochs for speed (or early stopping)
        for _ in range(5):
            _, _ = training_epoch(model, optimizer, loss_crit, train_loader, device)
            val_loss, val_acc = validation_epoch(model, loss_crit, val_loader, device)

            early_stop(val_acc, model)
            if early_stop.early_stop:
                print("Early stopping triggered.")
                break

        early_stop.restore(model)
        
        # Evaluate on validation fold
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                output = model(x_val)
                pred = torch.argmax(output, dim=1)
                correct += (pred == y_val).sum().item()
                total += y_val.size(0)

        fold_acc = correct / total
        accuracies.append(fold_acc)

    # Return average CV accuracy
    return sum(accuracies) / len(accuracies)


## Training

Define training and validation epochs

In [18]:
def training_epoch(model, optimizer, loss_crit, train_loader, device):
    model.train()
    avg_train_loss = 0
    correct_train = 0
    total_train = 0
    
    # Load batches and train
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)  # Move data to device
        optimizer.zero_grad()
        outputs = model(batch_x)

        # Compute loss
        loss = loss_crit(outputs, batch_y)
        loss.backward()
        optimizer.step()

        # Accumulate train loss
        avg_train_loss += loss.item() * batch_x.size(0)

        # Accumulate train accuracy
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == batch_y).sum().item()
        total_train += batch_y.size(0)
    
    # Average training loss and accuracy over batches
    avg_train_loss /= total_train
    avg_train_accuracy = correct_train / total_train

    return avg_train_loss, avg_train_accuracy

In [19]:
def validation_epoch(model, criterion, val_loader, device):
    model.eval()
    avg_val_loss = 0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad(): # No training
        for batch_x, batch_y in val_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)  # Move data to device
            outputs = model(batch_x)

            # Compute loss
            loss = criterion(outputs, batch_y)

            # Accumulate train loss
            avg_val_loss += loss.item() * batch_x.size(0)

            # Accumulate validation accuracy
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == batch_y).sum().item()
            total_val += batch_y.size(0)

    
    # Average validation loss for the fold
    avg_val_loss /= total_val
    avg_val_accuracy = correct_val / total_val
    
    return avg_val_loss, avg_val_accuracy

In [20]:
def train_model(
    model,
    df_train,
    vocab_inst,
    optimizer,
    loss_crit,
    device,
    hyperparameters,
    num_epochs=10,
    early_stopping=None
    ):

    train_df, val_df = train_test_split(
        df_train,
        test_size=val_size,
        stratify=train_val_df["Sentiment_encoded"],
        random_state=42
        )

    train_loader, val_loader, vocab_inst = create_dataloaders(train_df, val_df, vocab_inst=vocab_inst)

    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": []
        }

    early_stop = EarlyStopping(mode='max')

    for epoch in range(1, num_epochs + 1):
        print(f"Epoch {epoch}/{num_epochs}")

        # Training step
        train_loss, train_acc = training_epoch(model, optimizer, loss_crit, train_loader, device)

        # Validation step
        val_loss, val_acc = validation_epoch(model, loss_crit, val_loader, device)

        # Log metrics
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

        early_stop(val_acc, model)
        if early_stop.early_stop:
            early_stop.restore(model)
            print("Early stopping triggered.")
            break

    return model, history


## Evaluation

In [21]:
def evaluate_model_sklearn(model, dataloader, num_classes, device='cpu'):
    model.eval()
    all_preds = []
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            inputs, labels = batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            probs = outputs.detach().cpu().numpy()
            preds = np.argmax(probs, axis=1)

            all_probs.extend(probs)
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    all_probs = np.array(all_probs)
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    print("📊 Evaluation Results")
    print("-------------------------")
    print(f"Accuracy         : {accuracy_score(all_labels, all_preds):.4f}")
    print(f"Precision (macro): {precision_score(all_labels, all_preds, average='macro'):.4f}")
    print(f"Recall    (macro): {recall_score(all_labels, all_preds, average='macro'):.4f}")
    print(f"F1 Score  (macro): {f1_score(all_labels, all_preds, average='macro'):.4f}")
    print("\nConfusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

    # Binarize the labels for ROC computation
    y_true_bin = label_binarize(all_labels, classes=np.arange(num_classes))

    # Compute ROC curve and AUC for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], all_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute macro-average ROC curve and AUC
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))
    mean_tpr = np.zeros_like(all_fpr)

    for i in range(num_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    mean_tpr /= num_classes
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    print(f"\nROC-AUC (macro-average): {roc_auc['macro']:.4f}")

    # Plot all ROC curves
    plt.figure(figsize=(8, 6))
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], label=f"Class {i} (AUC = {roc_auc[i]:.2f})")
    plt.plot(fpr["macro"], tpr["macro"], label=f"Macro Avg (AUC = {roc_auc['macro']:.2f})", 
             linestyle='--', color='navy')
    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multiclass ROC Curves')
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

TODO multiple folds

# Main

## Import data and create dataframes

OS-agnostic working folder and data folder definition

In [22]:
CodeDirectory = Path(os.path.abspath(''))
DATASET_FOLDER = os.path.join(str(CodeDirectory.parent.absolute()), "_data","")
RAW_DATA_FOLDER = os.path.join(str(DATASET_FOLDER), "FinancialPhraseBank-v1.0","")

In [23]:
data_csv_path = DATASET_FOLDER+'data.csv'

Import data into pandas dataframe

In [24]:
df, le = create_df()

Please select which dataset to use. Data is categorized based on the percentage of agreement in the sentiment estimator.
Options:
(1) 50
(2) 66
(3) 75
(4) 100
/Users/exterior/Documents/IML/Project/Part2/_data/sentences_50.csv already exists. Skipping processing.


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4730 entries, 0 to 4729
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Sentence           4730 non-null   object
 1   Sentiment          4730 non-null   object
 2   Sentiment_encoded  4730 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 111.0+ KB


Split data into testing and training

In [26]:
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    stratify=df["Sentiment_encoded"],
    random_state=42
    )

## Preliminary analysis

## Train

### Hyperparameter search

In [27]:
vocab_inst = build_vocab(df_train)

Select device

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

Hyperparameter search

In [30]:
study = optuna.create_study(direction="maximize")  # You want to maximize accuracy
study.optimize(
    partial(objective, df=df, vocab_inst=vocab_inst, cv=skf, device=device),
    n_trials=5,  # Number of trials to try
)

[I 2025-06-13 16:24:24,526] A new study created in memory with name: no-name-0b4e238a-5ebd-4d32-b952-f7d02b46d074
/Users/exterior/miniconda3/envs/project2/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/exterior/miniconda3/envs/project2/lib/python3.12/site-packages/torch/nn/functional.py:1381: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn("dropout2d: Received a 3D input

Early stopping triggered.


/Users/exterior/miniconda3/envs/project2/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Early stopping triggered.
Early stopping triggered.


[I 2025-06-13 16:42:42,650] Trial 0 finished with value: 0.5953488010283184 and parameters: {'embed_dim': 64, 'lr': 0.0019348024511993806, 'weight_decay': 0.005692081828030703, 'lstm_out': 224, 'dropout_spatial': 0.30000000000000004, 'dropout_lstm': 0.5, 'dropout_1': 0.30000000000000004, 'dropout_2': 0.1, 'dense_1': 128}. Best is trial 0 with value: 0.5953488010283184.
/Users/exterior/miniconda3/envs/project2/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/exterior/miniconda3/envs/project2/lib/python3.12/site-packages/torch/nn/functional.py:1381: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This b

KeyboardInterrupt: 